In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
sys.path.insert(0, './gdrive/My Drive/module')

In [0]:
import pandas as pd
import numpy as np
import functions as f
import selfmadefunctions as sf

In [0]:
path = path = './gdrive/My Drive/data/'
meta = 'item_metadata.csv'
train = 'train.csv'
test = 'test.csv'
noinfo = 'noinfo_from_df_train.csv'
mono = 'df_mono.csv'
many = 'df_many.csv'
exsub = 'submission_popular.csv'

In [0]:
df_test = pd.read_csv(path + test)
df_train = pd.read_csv(path + train)
df_test_no = pd.read_csv(path + noinfo)


#1. df_test_noinfo 만들기

In [0]:
df_tar = f.get_submission_target(df_test)

In [0]:
df_tar_one = df_tar.loc[df_tar.step == 1]

In [0]:
print("the number of sessions with clickout step == 1 : ", df_tar_one.shape[0])
print("ratio of noinfo session in the target session : ", df_tar_one.shape[0]/df_tar.shape[0] * 100, "%")

the number of sessions with clickout step == 1 :  60605
ratio of noinfo session in the target session :  23.90041526503216 %


In [0]:
sess_no = df_tar_one.session_id

In [0]:
(df_test.loc[df_test.session_id.isin(sess_no)] == df_tar_one).all(axis = 0, skipna = True) # df_tar_one이 곧 df_test에서 noinfo session만 뽑아낸 것!

user_id             True
session_id          True
timestamp           True
step                True
action_type         True
reference          False
platform            True
city                True
device              True
current_filters    False
impressions         True
prices              True
dtype: bool

"step ==1에 clickout한 session_id들은 df_test에 clickout밖에 없다! (completely no previous info) "

In [0]:
df_tar_one.to_csv(path + 'noinfo_from_df_train.csv', index = False)

# 2. df_train + df_test concat해서(df_full) 'user_id' 기준으로 interaction 순으로sorting하기 

In [0]:
df_train_no = df_train.loc[df_train.user_id.isin(df_test_no.user_id)] # with user_id from 'df_test_no'

In [0]:
df_train_no.session_id.nunique()

9283

In [0]:
# concat
df_con = pd.concat([df_train, df_test])

In [0]:
df_con.sort_values(['user_id', 'step'], inplace = True) 

In [0]:
df_con = df_con.reset_index(drop = True)

In [0]:
# df_con.to_csv(path + 'full.csv', index = False)

In [0]:
df_tar = f.get_submission_target(df_con)

In [0]:
# df_con 재정의: df_tar의 user만 뽑아냄
df_con = df_con.loc[df_con.user_id.isin(df_tar.user_id)]

In [0]:
# df_con size 보자 
print('df_con의 shape: ',df_con.shape, '//df_con의 user_id unique 개수: ',df_con.user_id.nunique(), '//df_con의 session_id unique 개수: ', df_con.session_id.nunique())
df_con.groupby('user_id').session_id.count()

df_con의 shape:  (5050260, 12) //df_con의 user_id unique 개수:  250852 //df_con의 session_id unique 개수:  349334


user_id
000324D9BBUC     10
0004Q49X39PY      2
00071784XQ6B      2
0008BO33KUQ0     58
000GO9NY6P4M      6
000IRHJS2DL9     12
000JB0UNEH23      1
000OWRCYEHKT      3
000VBY1D6BP8      7
00130PO6XSZR      1
0013L641G5P2     20
0015X90DOWT1      3
0017FIR55K7R      8
001D536Q4S8D      1
001I0INS8EBP      7
001LT4KVZ14I      5
001P60T145X4      2
001TD1JLUE4D      9
001WP37I7I1T    316
001XGI2NI73R     13
0020N6R8BEDO     14
00210VAF61G5      3
0022L3757L7J    374
0023UTXOFLJP      3
0028DFS9FKIP      2
0029BRXGBS69     50
0029FEXX2R5F     25
002BISXP1U1Q      3
002CQHIQ78E8     22
002CUWLQDUV7      7
               ... 
ZZRMD5MI1L99     18
ZZS7YZDEKD9A      2
ZZSC8XZ6A8NP     14
ZZSHMZO6F36T      1
ZZSZN2E4ANXP      1
ZZTADJSKLQP8      1
ZZTFVH93H4ME      6
ZZTGL330L8UH      3
ZZUTJ8I05P4P      3
ZZV1ZO0IG4L2     18
ZZW4WCX342X6     46
ZZW77165VCI9     16
ZZWBXPVI1AA8      3
ZZWI88CNBB7C      6
ZZWT75PS0J6R     20
ZZWVBPY9GVOO      1
ZZWZBXI100ZW     25
ZZX3QP2D67V3      8
ZZX96IB781KD

In [0]:
# df_mono <- user_id 중에서 clickout이 한 번

def mono(df):
  s = (df.loc[df.action_type == 'clickout item'].groupby('user_id').action_type.count() == 1)
  mono_sess_index = s.index[s].to_list() # return session_id index
  df_mono = df.loc[df.user_id.isin(mono_sess_index)]
  
  return df_mono


df_mono = mono(df_con)

In [0]:
print(df_mono.shape)
df_mono.groupby('user_id').session_id.size() # user_id에서 clickout은 한 번이지만 session이 많은 경우가 많다.

(1147317, 12)


user_id
000324D9BBUC    10
000JB0UNEH23     1
00130PO6XSZR     1
0015X90DOWT1     3
001D536Q4S8D     1
001I0INS8EBP     7
001LT4KVZ14I     5
001P60T145X4     2
001TD1JLUE4D     9
00210VAF61G5     3
0023UTXOFLJP     3
0028DFS9FKIP     2
0029FEXX2R5F    25
002DU3SR91P5     1
002EOD1A549M     2
002H6CBQZ649    27
002N9ZH16N1R     1
002NTOHZ5ED7    78
002T8HU4LZAS     1
002XKX4GXZ4T     3
002YMA0MBZRU     1
0034995IC4NO     1
003A3JYSRGG8     1
003B331S7UDW     1
003F9HSS9A74     2
003QACWP50CH     3
003X06KWBDX5     5
00407X218KLB     2
00488C3ETUAP     1
0048PS2O6H4C     1
                ..
ZZGZIP2C9QHY    61
ZZHB75UM0BGO    34
ZZHGH47FAML8     1
ZZHUJL6WCXQK     1
ZZIKR683WHQL     2
ZZJHEJ663XEF    12
ZZJWIGQEMLUY     1
ZZKKPPAA61F8     3
ZZMPQ395DO4E     1
ZZMQSNA3X9NU     1
ZZN37EJFUKM6     1
ZZO3PB6GNFHV     1
ZZOYWHF1QY4H     2
ZZQP8L1GXPKM     4
ZZQZR8KSXBHW    17
ZZREITSO0Z1K     1
ZZRJ4VP7NV4B     1
ZZSHMZO6F36T     1
ZZSZN2E4ANXP     1
ZZTADJSKLQP8     1
ZZTGL330L8UH     3
ZZV1

In [0]:
# df_many <- user_id 중에서 clickout이 두 번 이상

def many(df):
  s = (df.loc[df.action_type == 'clickout item'].groupby('user_id').action_type.count() != 1)
  many_sess_index = s.index[s].to_list() # return session_id index
  df_many = df.loc[df.user_id.isin(many_sess_index)]
  
  return df_many

df_many = many(df_con)

In [0]:
print(df_many.shape)

(3902943, 12)


In [0]:
df_mono.to_csv(path + 'df_mono.csv', index = False)
df_many.to_csv(path + 'df_many.csv', index = False)


In [0]:
df_mono = pd.read_csv(path + mono)
df_many = pd.read_csv(path + many)

In [0]:
# 여기서부터!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! ref count
df_mono['reference_int'] = df_mono.reference.apply(sf.only_number)
df_many['reference_int'] = df_many.reference.apply(sf.only_number)

In [0]:
# only mono
df_mono_ref_count = sf.reference_count(df_mono)

In [0]:
df_mono_expl = f.explode(f.get_submission_target(df_mono), "impressions")

In [0]:
df_out = sf.ref_recommendation(df_mono_expl, df_mono_ref_count)

In [0]:
df_out

In [0]:
# df_many ref로 sorting
df_many_ref_count = sf.reference_count(df_many)
df_many_expl = f.explode(f.get_submission_target(df_many), "impressions")
df_out2 = sf.ref_recommendation(df_many_expl, df_many_ref_count)

In [14]:
df_out.shape[0] + df_out2.shape[0]

253573

In [0]:
sub = pd.concat([df_out, df_out2])

In [0]:
sub.sort_values(by = 'user_id', inplace = True)

In [0]:
sub.to_csv(path + '190515.csv', index = False)

# 3. [문제들]

## 3-1 train과 test에 중복되는 session_id
* 16개의 중복되는 session_id가 존재. 
* train과 test의 시간이 이어지지 않는다. 오히려 시간 간격이 크다!


In [0]:
# 1. train과 test에 중복되는 session_id 존재하는가? yes
dup = set(df_train.session_id) & set(df_test.session_id)

In [0]:
# timestamp(UNIX)를 local date, time으로 변환 
import datetime
def myfunc(x):
    return datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')

# ex. train['date'] = list(map(myfunc,list(train.timestamp)))


In [0]:
# 2. 그 session은 무엇인가? -> theory: 시간순으로 끊어졌겠지? NO!! 시간 간격이 큰데?? 왜 session duplicate가 존재하는 거지?
tempt_train = df_train.loc[df_train.session_id.isin(dup)]
tempt_test = df_test.loc[df_test.session_id.isin(dup)]

tempt_train['time'] = list(map(myfunc,list(tempt_train.timestamp)))
tempt_test['time'] = list(map(myfunc,list(tempt_test.timestamp)))

for sess in dup:
  print(tempt_train.loc[tempt_train.session_id == sess, 'time'], '\n', tempt_test.loc[tempt_test.session_id == sess, 'time'])
  print('#######################################################################################')


501300    2018-11-01 23:30:43
501301    2018-11-01 23:32:44
Name: time, dtype: object 
 2994018    2018-11-07 03:50:59
2994019    2018-11-07 03:51:18
2994020    2018-11-07 03:51:43
2994021    2018-11-07 03:51:43
2994022    2018-11-07 03:51:43
2994023    2018-11-07 03:51:53
2994024    2018-11-07 03:52:05
Name: time, dtype: object
#######################################################################################
13725354    2018-11-06 21:53:01
13725355    2018-11-06 21:56:56
13725356    2018-11-06 21:57:06
13725357    2018-11-06 21:57:06
13725358    2018-11-06 21:57:06
13725359    2018-11-06 21:57:06
13725360    2018-11-06 21:57:57
13725361    2018-11-06 22:02:24
13725362    2018-11-06 22:02:24
13725363    2018-11-06 22:06:24
13725364    2018-11-06 22:06:34
13725365    2018-11-06 22:06:42
13725366    2018-11-06 22:15:38
13725367    2018-11-06 22:15:48
13725368    2018-11-06 22:15:57
13725369    2018-11-06 22:18:20
13725370    2018-11-06 22:20:06
13725371    2018-11-06 22:20:16
13725

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


## 3-2 train과 test에 중복되는 user_id 
* previously -> test의 session을 기반으로 정보를 구했다.
* previously -> train + test concat해서 써야하나? -> user_id기반
* 실험 1 : train + test concat해서 ref기반 + cossim 기반 구해서 값 score 확인 (noinfo session 중에서 값이 채워지는 경우 생긴다.) > *실험 결과: df_concat, user_id 기반이 맞다!!*

---


* 향후 고민거리: train + test concat에서 clickout만 있는 경우 (완전 noinfo) 어떻게 sorting해야 하는가 > *collaborative filtering model , RNN 적용*

In [0]:
# train과 test의 user_id 중복 개수?
print("df_train에 user_id 개수: ",len(set(df_train.user_id)), "/ df_test에 user_id 개수: ", len(set(df_test.user_id)), '\n 그중에 중복되는 user_id 개수: ', len(set(df_train.user_id) & set(df_test.user_id)))

df_train에 user_id 개수:  730803 / df_test에 user_id 개수:  250852 
 그중에 중복되는 user_id 개수:  33614
